In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import requests
import csv
import json


In [ ]:
# ALHareem website
from google.colab import drive
drive.mount('/content/drive')
class Scraper:
    def __init__(self, base_url, folder_path, csv_file_path, max_pages,category):
        # Initialize WebDriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(options=chrome_options)

        # Store base URL and file paths
        self.base_url = base_url
        self.folder_path = folder_path
        os.makedirs(self.folder_path, exist_ok=True)  # Create folder if it doesn't exist
        self.csv_file_path = csv_file_path
        self.max_pages = max_pages
        self.category=category
        self.data = []

    def extract_data(self):
        page_number = 1
        while page_number <= self.max_pages:
            self.driver.get(f"https://hijabulhareem.com/collections/abaya-jilbab?page={page_number}")
            time.sleep(5)

            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            items = soup.find("div", {"class": "on_list_view_false products nt_products_holder row fl_center row_pr_2 cdt_des_1 round_cd_false nt_contain ratio_nt position_8 space_30 nt_default"})

            print(f"Number of items found on page {page_number}: {len(items)}")

            if not items:
                print("No items found. Ending pagination.")
                break

            for idx, item in enumerate(items):
              try:
                # Extract product name
                name_tag = item.find("h3", class_="product-title pr fs__14 mg__0 fwm")
                product_name = name_tag.find("a").get_text(strip=True) if name_tag else None


                # Extract product price
                price = round(float(item.select_one("span.price").text.replace("Rs.","").replace(",","")) / 230, 2)




                # Extract product ID
                product_id_tag = item.find("div", attrs={"data-id": True})
                product_id = product_id_tag["data-id"] if product_id_tag else None


                #Extract product link
                product_link = item.select_one("h3.product-title a")["href"]
                product_link = "https://hijabulhareem.com" + product_link


                # p_img (أعلى جودة)
                img = item.select_one("div.pr_lazy_img.main-img")
                bgset = img["data-bgset"]
                p_img = bgset.split(",")[-1].split()[0]
                if p_img.startswith("//"):
                  image_url = "https:" + p_img
                else:
                  image_url = p_img


                if product_id and product_name and price and image_url and product_link:
                  # Store extracted data
                  self.data.append({
                  'p_id': product_id,
                  'source': 'AlHreem',
                  'p_base_price': price,
                  'p_name': product_name,
                  'category': self.category,
                  'p_gender': 'Women',
                  'p_img': image_url,
                  'p_link': product_link})

                  print(f"Page {page_number}, Item {idx}: Image URL: {image_url}, Price: {price}, Name: {product_name}")
                else:
                  print(f"Item {idx} on page {page_number} skipped due to missing data.")
              except :
                pass

            page_number += 1


    def save_data_to_csv(self):
        df = pd.DataFrame(self.data)
        if os.path.exists(self.csv_file_path):
            df.to_csv(self.csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(self.csv_file_path, index=False)
        print("Data saved to CSV.")

    def close(self):
        self.driver.quit()


folder_path = '/content/drive/My Drive/Abayas'
csv_file_path = '/content/drive/My Drive/Abayas/Abayas_dataset5.csv'

categories={
    'Abayas':6

}
for key,value in categories.items():

  url=f"https://hijabulhareem.com/collections/abaya-jilbab?page={value}"
  scraper = Scraper(url, folder_path, csv_file_path,value,key)
  scraper.extract_data()
  scraper.save_data_to_csv()
  scraper.close()



In [ ]:
# Noir Website
from google.colab import drive
drive.mount('/content/drive')
class Scraper:
    def __init__(self, base_url, folder_path, csv_file_path, max_pages,category):
        # Initialize WebDriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(options=chrome_options)

        # Store base URL and file paths
        self.base_url = base_url
        self.folder_path = folder_path
        os.makedirs(self.folder_path, exist_ok=True)  # Create folder if it doesn't exist
        self.csv_file_path = csv_file_path
        self.max_pages = max_pages
        self.category=category
        self.data = []

    def extract_data(self):
        page_number = 1
        while page_number <= self.max_pages:
            self.driver.get(f"https://abayanoir.com/product-category/abaya/page/{page_number}/?v=fbe46383db39")
            time.sleep(5)

            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            items = soup.find("div", {"class": "products wd-products wd-grid-g grid-columns-3 elements-grid pagination-pagination title-line-two wd-stretch-cont-lg wd-products-with-shadow"})

            print(f"Number of items found on page {page_number}: {len(items)}")

            if not items:
                print("No items found. Ending pagination.")
                break

            for idx, item in enumerate(items):
              try:
                # Extract product name
                title_tag = item.find("h3", class_="wd-entities-title")
                if title_tag:
                   product_name = title_tag.get_text(strip=True)
                else:
                   product_name = None

                # Extract product price
                price_tag = item.find("span", class_="woocommerce-Price-amount amount")
                price = price_tag.get_text(strip=True) if price_tag else None


                # Extract product ID
                product_id = item.get("data-id")

                # Extract image
                image_url = soup.find("img")["src"]

                # Ensure full URL (add "https:" if missing)
                image_tag = item.find("img")
                if image_tag:
                   if "srcset" in image_tag.attrs:
                       image_url = image_tag["srcset"].split(",")[0].split(" ")[0].strip()
                   else:
                       image_url = image_tag["src"]
                else:
                   image_url = None

                # Extract product link
                try:
                  product_link = item.find("a")["href"]   # Direct link from the <a> tag
                  print("Product Link:", product_link)
                except:
                  product_link =None


                if product_id and product_name and price and image_url and product_link:
                  # Store extracted data
                  self.data.append({
                  'p_id': product_id,
                  'source': 'Noir',
                  'p_base_price': price,
                  'p_name': product_name,
                  'category': self.category,
                  'p_gender': 'Women',
                  'p_img': image_url,
                  'p_link': product_link})

                  print(f"Page {page_number}, Item {idx}: Image URL: {image_url}, Price: {original_price}, Name: {product_name}")
                else:
                  print(f"Item {idx} on page {page_number} skipped due to missing data.")
              except :
                pass

            page_number += 1


    def save_data_to_csv(self):
        df = pd.DataFrame(self.data)
        if os.path.exists(self.csv_file_path):
            df.to_csv(self.csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(self.csv_file_path, index=False)
        print("Data saved to CSV.")

    def close(self):
        self.driver.quit()

# if you want any others categories change url and no. of pages
folder_path = '/content/drive/My Drive/Abayas'
csv_file_path = '/content/drive/My Drive/Abayas/Abayas_dataset.csv'

categories={
    'Abayas':6

}
for key,value in categories.items():

  url=f"https://abayanoir.com/product-category/abaya/page/{value}/?v=fbe46383db39"
  scraper = Scraper(url, folder_path, csv_file_path,value,key)
  scraper.extract_data()
  scraper.save_data_to_csv()
  scraper.close()



In [ ]:
# Dhaagy wbsite

from google.colab import drive
drive.mount('/content/drive')
class Scraper:
    def __init__(self, base_url, folder_path, csv_file_path, max_pages,category):
        # Initialize WebDriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(options=chrome_options)

        # Store base URL and file paths
        self.base_url = base_url
        self.folder_path = folder_path
        os.makedirs(self.folder_path, exist_ok=True)  # Create folder if it doesn't exist
        self.csv_file_path = csv_file_path
        self.max_pages = max_pages
        self.category=category
        self.data = []

    def extract_data(self):
        page_number = 1
        while page_number <= self.max_pages:
            self.driver.get(f"https://www.dhaagey.pk/collections/abayas?page={page_number}")
            time.sleep(5)

            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            items = soup.find("div", {"class": "m-collection-products m:flex m:flex-wrap m-cols-4"})

            print(f"Number of items found on page {page_number}: {len(items)}")

            if not items:
                print("No items found. Ending pagination.")
                break

            for idx, item in enumerate(items):
              try:
                # Extract product name
                h3_tag = item.find("h3", class_="m-product-card__title")
                if h3_tag:
                  a_tag = h3_tag.find("a")
                  product_name = a_tag.get_text(strip=True) if a_tag else None
                else:
                  product_name = None


                # Extract product price
                price_text = item.find("span", class_="m-price-item--last").get_text(strip=True)
                # إزالة العلامة Rs وأي فواصل، وتحويل الرقم لفلوات
                price_number = float(price_text.replace("Rs.","").replace(",",""))
                # تحويل للعملة (مثلاً لو 1 دولار = 82 روبية هندية)
                price = round(price_number / 82, 2)

                #Extract ID
                product_id_div = item.find("div", class_="m-product-card")
                if not product_id_div:
                    product_id_div = item
                product_id = product_id_div.get("data-product-id")
                print("Product ID:", product_id)

                # Extract main product image
                image_div = item.find("div", class_="m-product-card__main-image")
                if image_div:
                    img_tag = image_div.find("img")
                    if img_tag and img_tag.get("src"):
                       image_url = img_tag["src"]
                       if image_url.startswith("//"):
                          image_url = "https:" + image_url
                       print("Image Link:", image_url)
                    else:
                       print("No image found")
                else:
                    print("No image div found")

                # Extract product link
                p_link = item.find("a", class_="m-product-card__link")["href"]
                product_link = "https://www.dhaagey.pk" + p_link



                if product_id and product_name and price and image_url and product_link:
                  # Store extracted data
                  self.data.append({
                  'p_id': product_id,
                  'source': 'Dhaagy',
                  'p_base_price': price,
                  'p_name': product_name,
                  'category': self.category,
                  'p_gender': 'Women',
                  'p_img': image_url,
                  'p_link': product_link})

                  print(f"Page {page_number}, Item {idx}: Image URL: {image_url}, Price: {price}, Name: {product_name}")
                else:
                  print(f"Item {idx} on page {page_number} skipped due to missing data.")
              except :
                pass

            page_number += 1


    def save_data_to_csv(self):
        df = pd.DataFrame(self.data)
        if os.path.exists(self.csv_file_path):
            df.to_csv(self.csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(self.csv_file_path, index=False)
        print("Data saved to CSV.")

    def close(self):
        self.driver.quit()

# if you want any others categories change url and no. of pages
folder_path = '/content/drive/My Drive/Abayas'
csv_file_path = '/content/drive/My Drive/Abayas/Abayas_dataset3.csv'

categories={
    'Abayas':5

}
for key,value in categories.items():

  url=f"https://www.dhaagey.pk/collections/abayas?page={value}"
  scraper = Scraper(url, folder_path, csv_file_path,value,key)
  scraper.extract_data()
  scraper.save_data_to_csv()
  scraper.close()



In [ ]:
# malbus website

from google.colab import drive
drive.mount('/content/drive')
class Scraper:
    def __init__(self, base_url, folder_path, csv_file_path, max_pages,category):
        # Initialize WebDriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(options=chrome_options)

        # Store base URL and file paths
        self.base_url = base_url
        self.folder_path = folder_path
        os.makedirs(self.folder_path, exist_ok=True)  # Create folder if it doesn't exist
        self.csv_file_path = csv_file_path
        self.max_pages = max_pages
        self.category=category
        self.data = []

    def extract_data(self):
        page_number = 1
        while page_number <= self.max_pages:
            self.driver.get(f"https://www.bymalbus.com/collections/abaya?page={page_number}")
            time.sleep(5)

            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            items = soup.find("div", {"class": "collection__grid"})

            print(f"Number of items found on page {page_number}: {len(items)}")

            if not items:
                print("No items found. Ending pagination.")
                break

            for idx, item in enumerate(items):
              try:
                # Extract product name
                a_tag = item.find("h3", class_="card__title").find("a") if item.find("h3", class_="card__title") else None
                product_name = a_tag.get_text(strip=True) if a_tag else None


                # Extract product price
                price_tag = item.find("span", class_="money bacurr-money")
                price = price_tag.get_text(strip=True) if price_tag else None


                # Extract image
                image_url = soup.find("img")["src"]

                image_url = (item.find("img", class_="card__img card__img--hover")
                .get("srcset", "")
                .split(",")[-1]
                .split()[0]
                .replace("//", "https://"))


                # Extract product link
                link_tag = item.find("a", class_="card card--left")
                product_link = link_tag.get("href") if link_tag else None

                # لو الرابط relative (" /products/... ") نحوله لرابط كامل
                if product_link and product_link.startswith("/"):
                     product_link = "https://www.bymalbus.com" + product_link


                if product_name and price and image_url and product_link:
                  # Store extracted data
                  self.data.append({

                  'source': 'malbus',
                  'p_base_price': price,
                  'p_name': product_name,
                  'category': self.category,
                  'p_gender': 'Women',
                  'p_img': image_url,
                  'p_link': product_link})

                  print(f"Page {page_number}, Item {idx}: Image URL: {image_url}, Price: {price}, Name: {product_name}")
                else:
                  print(f"Item {idx} on page {page_number} skipped due to missing data.")
              except :
                pass

            page_number += 1


    def save_data_to_csv(self):
        df = pd.DataFrame(self.data)
        if os.path.exists(self.csv_file_path):
            df.to_csv(self.csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(self.csv_file_path, index=False)
        print("Data saved to CSV.")

    def close(self):
        self.driver.quit()

# if you want any others categories change url and no. of pages
folder_path = '/content/drive/My Drive/Abayas'
csv_file_path = '/content/drive/My Drive/Abayas/Abayas_dataset2.csv'

categories={
    'Abayas':3

}
for key,value in categories.items():

  url=f"https://www.bymalbus.com/collections/abaya?page={value}"
  scraper = Scraper(url, folder_path, csv_file_path,value,key)
  scraper.extract_data()
  scraper.save_data_to_csv()
  scraper.close()



In [ ]:
# Zeinab Butterfly website

from google.colab import drive
drive.mount('/content/drive')
class Scraper:
    def __init__(self, base_url, folder_path, csv_file_path, max_pages,category):
        # Initialize WebDriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(options=chrome_options)

        # Store base URL and file paths
        self.base_url = base_url
        self.folder_path = folder_path
        os.makedirs(self.folder_path, exist_ok=True)  # Create folder if it doesn't exist
        self.csv_file_path = csv_file_path
        self.max_pages = max_pages
        self.category=category
        self.data = []

    def extract_data(self):
        page_number = 1
        while page_number <= self.max_pages:
            self.driver.get(f"https://zainabcollection.com/product-category/butterfly-abaya/page/{page_number}/")
            time.sleep(5)

            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            items = soup.find("div", {"class": "products wd-products wd-grid-g grid-columns-3 elements-grid pagination-pagination"})

            print(f"Number of items found on page {page_number}: {len(items)}")

            if not items:
                print("No items found. Ending pagination.")
                break

            for idx, item in enumerate(items):
              try:
                # Extract product name
                name_tag = item.find("h3", class_="wd-entities-title")
                product_name = name_tag.find("a").get_text(strip=True) if name_tag else None


                # Extract product price
                price_tag = item.find("span", class_="price")
                price_rs = price_tag.get_text(strip=True) if price_tag else None

                if price_rs:
                  # حذف الرمز والفواصل وتحويل النص لرقم
                  price_number = float(price_rs.replace("₨", "").replace("\xa0", "").replace(",", "").strip())
                  # التحويل للدولار
                  conversion_rate = 230  # مثال: 1 دولار ≈ 230 Rs
                  price = round(price_number / conversion_rate, 2)


                # Extract product ID
                product_id = item.get("data-id")  # من div الرئيسي



                # Extract image URL
                img_tag = item.find("img")
                image_url = img_tag.get("data-src") or img_tag.get("src") if img_tag else None

                # Ensure full URL (add "https:" if missing)
                if image_url.startswith("//"):
                     image_url = "https:" + image_url
                     print("Image Link:", image_url)
                # Extract product link
                link_tag = item.find("a", class_="product-image-link")
                product_link = link_tag.get("href") if link_tag else None




                if product_id and product_name and price and image_url and product_link:
                  # Store extracted data
                  self.data.append({
                  'p_id': product_id,
                  'source': 'Butterfly',
                  'p_base_price': price,
                  'p_name': product_name,
                  'category': self.category,
                  'p_gender': 'Women',
                  'p_img': image_url,
                  'p_link': product_link})

                  print(f"Page {page_number}, Item {idx}: Image URL: {image_url}, Price: {price}, Name: {product_name}")
                else:
                  print(f"Item {idx} on page {page_number} skipped due to missing data.")
              except :
                pass

            page_number += 1


    def save_data_to_csv(self):
        df = pd.DataFrame(self.data)
        if os.path.exists(self.csv_file_path):
            df.to_csv(self.csv_file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(self.csv_file_path, index=False)
        print("Data saved to CSV.")

    def close(self):
        self.driver.quit()

# if you want any others categories change url and no. of pages
folder_path = '/content/drive/My Drive/Abayas'
csv_file_path = '/content/drive/My Drive/Abayas/Abayas_dataset4.csv'

categories={
    'Abayas':5

}
for key,value in categories.items():

  url=f"https://zainabcollection.com/product-category/butterfly-abaya/page/{value}/"
  scraper = Scraper(url, folder_path, csv_file_path,value,key)
  scraper.extract_data()
  scraper.save_data_to_csv()
  scraper.close()

